In [1]:
!pip install wandb -qU

In [2]:
!pip install nltk gensim contractions

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: gozdeunveer (tumnlp). Use `wandb login --relogin` to force relogin


True

## Import data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/multilingual-detection-of-persuasion-techniques-in-memes

/content/drive/MyDrive/multilingual-detection-of-persuasion-techniques-in-memes


In [6]:
import json
train_input_file = open("data/subtask1/train.json")
val_input_file=open("data/subtask1/validation.json")
train_raw_data=json.load(train_input_file)
val_raw_data=json.load(val_input_file)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download("punkt")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
import contractions
from nltk.tokenize import word_tokenize

# from https://www.kaggle.com/code/lystdo/lstm-with-word2vec-embeddings/script
def clean_text(text,remove_stopwords=False, stem_words=False):


  text= text.replace("\\n", " ")
  translator = str.maketrans("", "", punctuation)
  text = text.translate(translator)

  text = text.lower().split()
  text=[ contractions.fix(w) for w in text]
  # Optionally, remove stop words
  if remove_stopwords:
      stops = set(stopwords.words("english"))
      text = [w for w in text if not w in stops]

  text = " ".join(text)

  # Clean the text


  text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r",", " ", text)
  text = re.sub(r"\.", " ", text)
  text = re.sub(r"!", " ! ", text)
  text = re.sub(r"\/", " ", text)
  text = re.sub(r"\^", " ^ ", text)
  text = re.sub(r"\+", " + ", text)
  text = re.sub(r"\-", " - ", text)
  text = re.sub(r"\=", " = ", text)
  text = re.sub(r"'", " ", text)
  text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
  text = re.sub(r":", " : ", text)
  text = re.sub(r" e g ", " eg ", text)
  text = re.sub(r" b g ", " bg ", text)
  text = re.sub(r" u s ", " american ", text)
  text = re.sub(r"\0s", "0", text)
  text = re.sub(r" 9 11 ", "911", text)
  text = re.sub(r"e - mail", "email", text)
  text = re.sub(r"j k", "jk", text)
  text = re.sub(r"\s{2,}", " ", text)


  if stem_words:
    sb = SnowballStemmer("english")
    text = text.split()
    stemmed_words = [sb.stem(word) for word in text]
    text = " ".join(stemmed_words)

  lemmatizer = WordNetLemmatizer()
  text = text.split()
  lem_sent = [lemmatizer.lemmatize(words_sent) for words_sent in text]

  text = " ".join(lem_sent)

    # Return a list of words
  return word_tokenize(text)


In [9]:
import pandas as pd
from nltk.tokenize import word_tokenize
df_train=pd.DataFrame(train_raw_data)

df_train['text'] = df_train['text'].map(lambda x: clean_text(x,False,False))
df_val=pd.DataFrame(val_raw_data)

df_val['text'] = df_val['text'].map(lambda x: clean_text(x,False,False))
df_train

,id,text,labels,link
0,65635,"[this, is, why, you, need, a, sharpie, with, y...",[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/photo/?fbid=402355213...
1,67927,"[good, news, nazanin, zaghariratcliffe, and, a...","[Loaded Language, Glittering generalities (Vir...",https://www.facebook.com/amnesty/photos/531198...
2,68031,"[paing, phyo, min, is, free]",[],https://www.facebook.com/amnesty/photos/427419...
3,77490,"[move, your, ship, away, oooook, move, your, s...",[],https://www.facebook.com/rightpatriots/photos/...
4,67641,"[when, you, are, the, fbi, they, let, you, do,...",[Thought-terminating cliché],https://www.facebook.com/AddictingInfoOrg/phot...
...,...,...,...,...
6995,67360,"[if, your, doctor, prescribes, you, medication...","[Loaded Language, Causal Oversimplification, T...",https://www.facebook.com/TheControversia/photo...
6996,70579,"[defends, trump, made, allegation, of, electio...","[Loaded Language, Whataboutism]",https://www.facebook.com/PatriotFetch/photos/p...
6997,70305,"[i, am, having, trouble, selling, our, incredi...",[],https://www.facebook.com/PatriotFetch/photos/p...
6998,77769,"[i, am, so, happy, we, live, in, a, world, wit...",[],https://www.facebook.com/communism101/photos/5...


In [10]:
df_train=df_train[df_train['labels'].map(lambda d: len(d)) > 0]
df_val=df_val[df_val['labels'].map(lambda d: len(d)) > 0]

unique_labels=df_train["labels"].explode().unique()
int_map=range(len(unique_labels))
int_to_label=dict(zip(int_map,unique_labels))
labels_to_int=dict(zip(unique_labels,int_map))

def transform_labels(labels):
  result=[]
  for el in labels:
    result.append(labels_to_int[el])
  return result

df_train["labels_int"]=df_train["labels"].apply(transform_labels)
df_val["labels_int"]=df_val["labels"].apply(transform_labels)

df_train

<ipython-input-10-f20a5fc3e70f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["labels_int"]=df_train["labels"].apply(transform_labels)


,id,text,labels,link,labels_int
0,65635,"[this, is, why, you, need, a, sharpie, with, y...",[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/photo/?fbid=402355213...,[0]
1,67927,"[good, news, nazanin, zaghariratcliffe, and, a...","[Loaded Language, Glittering generalities (Vir...",https://www.facebook.com/amnesty/photos/531198...,"[1, 2]"
4,67641,"[when, you, are, the, fbi, they, let, you, do,...",[Thought-terminating cliché],https://www.facebook.com/AddictingInfoOrg/phot...,[3]
6,79204,"[january, 2021, you, do, not, buy, a, 12, mill...",[Whataboutism],https://www.facebook.com/ResistanceFeed/photos...,[4]
7,79372,"[term, limit, are, everywhere, politician, can...",[Slogans],null,[5]
...,...,...,...,...,...
6993,65103,"[me, waiting, for, trump, to, be, held, accoun...",[Smears],https://www.facebook.com/1640832309490921/phot...,[7]
6994,71170,"[it, time, for, congress, to, do, it, job, and...","[Appeal to authority, Black-and-white Fallacy/...",null,"[9, 0, 1]"
6995,67360,"[if, your, doctor, prescribes, you, medication...","[Loaded Language, Causal Oversimplification, T...",https://www.facebook.com/TheControversia/photo...,"[1, 6, 3, 8]"
6996,70579,"[defends, trump, made, allegation, of, electio...","[Loaded Language, Whataboutism]",https://www.facebook.com/PatriotFetch/photos/p...,"[1, 4]"


In [34]:
unique_labels_val=df_val["labels"].explode().unique()

In [36]:
set(unique_labels_val)==set(unique_labels)

True

In [12]:
print(labels_to_int)

{'Black-and-white Fallacy/Dictatorship': 0, 'Loaded Language': 1, 'Glittering generalities (Virtue)': 2, 'Thought-terminating cliché': 3, 'Whataboutism': 4, 'Slogans': 5, 'Causal Oversimplification': 6, 'Smears': 7, 'Name calling/Labeling': 8, 'Appeal to authority': 9, 'Exaggeration/Minimisation': 10, 'Repetition': 11, 'Flag-waving': 12, 'Appeal to fear/prejudice': 13, 'Reductio ad hitlerum': 14, 'Doubt': 15, "Misrepresentation of Someone's Position (Straw Man)": 16, 'Obfuscation, Intentional vagueness, Confusion': 17, 'Bandwagon': 18, 'Presenting Irrelevant Data (Red Herring)': 19}


In [11]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(window=5,
    min_count=3,
    workers=4)
w2v_model.build_vocab(df_train["text"], progress_per=500)
w2v_model.train(df_train["text"], total_examples=w2v_model.corpus_count, epochs=w2v_model.epochs)
wv=w2v_model.wv

In [12]:
import numpy as np
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [13]:
df_train

,id,text,labels,link,labels_int
0,65635,"[this, is, why, you, need, a, sharpie, with, y...",[Black-and-white Fallacy/Dictatorship],https://www.facebook.com/photo/?fbid=402355213...,[0]
1,67927,"[good, news, nazanin, zaghariratcliffe, and, a...","[Loaded Language, Glittering generalities (Vir...",https://www.facebook.com/amnesty/photos/531198...,"[1, 2]"
4,67641,"[when, you, are, the, fbi, they, let, you, do,...",[Thought-terminating cliché],https://www.facebook.com/AddictingInfoOrg/phot...,[3]
6,79204,"[january, 2021, you, do, not, buy, a, 12, mill...",[Whataboutism],https://www.facebook.com/ResistanceFeed/photos...,[4]
7,79372,"[term, limit, are, everywhere, politician, can...",[Slogans],null,[5]
...,...,...,...,...,...
6993,65103,"[me, waiting, for, trump, to, be, held, accoun...",[Smears],https://www.facebook.com/1640832309490921/phot...,[7]
6994,71170,"[it, time, for, congress, to, do, it, job, and...","[Appeal to authority, Black-and-white Fallacy/...",null,"[9, 0, 1]"
6995,67360,"[if, your, doctor, prescribes, you, medication...","[Loaded Language, Causal Oversimplification, T...",https://www.facebook.com/TheControversia/photo...,"[1, 6, 3, 8]"
6996,70579,"[defends, trump, made, allegation, of, electio...","[Loaded Language, Whataboutism]",https://www.facebook.com/PatriotFetch/photos/p...,"[1, 4]"


In [14]:
df_train['sentence_vec'] = df_train['text'].apply(sent_vec)
df_val['sentence_vec'] = df_val['text'].apply(sent_vec)


X_train = df_train['sentence_vec'].to_list()
Y_train = df_train['labels_int'].to_list()
X_val = df_val['sentence_vec'].to_list()
Y_val = df_val['labels_int'].to_list()

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Y_train_transformed=MultiLabelBinarizer().fit_transform(Y_train)



<ipython-input-14-4e9cac509c31>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['sentence_vec'] = df_train['text'].apply(sent_vec)


## With logistic regression

In [20]:
clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, Y_train_transformed)
preds=clf.predict(X_val)

from sklearn.metrics import hamming_loss, accuracy_score,precision_score,recall_score
Y_val_transformed=MultiLabelBinarizer().fit_transform(Y_val)
acc=accuracy_score(Y_val_transformed, preds)
hamming_loss=hamming_loss(Y_val_transformed, preds)
precision_score=precision_score(Y_val_transformed, preds,average=None)
recall_score=recall_score(Y_val_transformed, preds,average=None)

print("Accuracy:",acc)
print("Hamming loss:",hamming_loss)
print("Logistic Regression Precision:",precision_score)
print("Logistic Regression Recall:",recall_score)

run="word2vec_logRegression_multilabel"
wandb.init(
      # Set entity to private name or teamname
      entity="tumnlp",
      # Set the project where this run will be logged
      project="visualize_sklearn",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      )
preds_prob=clf.predict_proba(X_val)
try:
  wandb.sklearn.plot_classifier(clf, X_train, X_val, Y_train_transformed, Y_val_transformed,preds,preds_prob,unique_labels)
# Mark the run as finished
except:
  wandb.log({"acc": acc, "hamming_loss": hamming_loss,"precision_score":precision_score,"recall_score":recall_score})
  wandb.finish()
wandb.finish()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.0
Hamming loss: 0.10194174757281553
Logistic Regression Precision: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Logistic Regression Recall: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


wandb: 
wandb: Plotting Classifier.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.


acc,▁
hamming_loss,▁
acc,0.0
hamming_loss,0.10194


## With random forest

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import hamming_loss, accuracy_score,precision_score,recall_score

clf = MultiOutputClassifier(RandomForestClassifier(max_depth=500)).fit(X_train, Y_train_transformed)
preds=clf.predict(X_val)


acc=accuracy_score(Y_val_transformed, preds)
hamming_loss=hamming_loss(Y_val_transformed, preds)
precision_score=precision_score(Y_val_transformed, preds,average="weighted")
recall_score=recall_score(Y_val_transformed, preds,average="weighted")

print("Accuracy:",acc)
print("Hamming loss:",hamming_loss)
print("Logistic Regression Precision:",precision_score)
print("Logistic Regression Recall:",recall_score)

run="word2vec_randomForest_multilabel_depth500"
wandb.init(
      # Set entity to private name or teamname
      entity="tumnlp",
      # Set the project where this run will be logged
      project="visualize_sklearn",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      config={"max_depth":500}
      )
preds_prob=clf.predict_proba(X_val)
try:
  wandb.sklearn.plot_classifier(clf, X_train, X_val, Y_train_transformed, Y_val_transformed,preds,preds_prob,unique_labels)
# Mark the run as finished
except:
  wandb.log({"acc": acc, "hamming_loss": hamming_loss,"precision_score_weighted":precision_score,"recall_score_weighted":recall_score})
  wandb.finish()
wandb.finish()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.03155339805825243
Hamming loss: 0.1046116504854369
Logistic Regression Precision: 0.434785914254828
Logistic Regression Recall: 0.06793802145411204


wandb: 
wandb: Plotting Classifier.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.


acc,▁
hamming_loss,▁
precision_score_weighted,▁
recall_score_weighted,▁
acc,0.03155
hamming_loss,0.10461
precision_score_weighted,0.43479
recall_score_weighted,0.06794


In [26]:

# Mark the run as finished
wandb.finish()